In [12]:
import os
import arcpy
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
gdb = r"Q:\projects\Mwlrs\AEB_Prioritization_Tool\data\AEB_Analysis.gdb"
arcpy.env.workspace= gdb

In [7]:
# Get data from the featureclass
fc= 'r20250117_AEB_Disturbance_Feature_Class'

fields = ["ASSESSMENT_UNIT_SOURCE_ID", "DISTRB_HUMAN_CRNT_20YR_PCNT", "RD_STRM_XING_DENS_CLS", "RD_STEEP_SLOPE_DENS_CLS", "RD_STRM_BUFF_100_DENS_WGHTD_CLS"]

data = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        ASSESSMENT_UNIT_SOURCE_ID= row[0]
        DISTRB_HUMAN_CRNT_20YR_PCNT = row[1]
        RD_STRM_XING_DENS_CLS = row[2]
        RD_STEEP_SLOPE_DENS_CLS = row[3]
        RD_STRM_BUFF_100_DENS_WGHTD_CLS = row[4]

        data.append([ASSESSMENT_UNIT_SOURCE_ID, DISTRB_HUMAN_CRNT_20YR_PCNT, RD_STRM_XING_DENS_CLS, RD_STEEP_SLOPE_DENS_CLS, RD_STRM_BUFF_100_DENS_WGHTD_CLS])

        rowcount += 1

In [8]:
# create a df for raw data
df = pd.DataFrame(data, columns=fields)
df.head()

,ASSESSMENT_UNIT_SOURCE_ID,DISTRB_HUMAN_CRNT_20YR_PCNT,RD_STRM_XING_DENS_CLS,RD_STEEP_SLOPE_DENS_CLS,RD_STRM_BUFF_100_DENS_WGHTD_CLS
0,18089.0,3.591746,Low,Low,Low
1,18090.0,8.152684,Moderate,Low,Low
2,18091.0,1.051743,Low,Low,Low
3,18086.0,4.595927,Moderate,Low,Low
4,18087.0,6.057131,Moderate,Low,Low


In [9]:
#Create RD_DENS_CLS
def classify_row(row):
    # Collect the three column values
    values = [
        row['RD_STRM_XING_DENS_CLS'], 
        row['RD_STEEP_SLOPE_DENS_CLS'], 
        row['RD_STRM_BUFF_100_DENS_WGHTD_CLS']
    ]
    
    # Count how many fields are 'High' and 'Moderate'
    high_count = values.count('High')
    moderate_count = values.count('Moderate')
    
    # Apply the classification rules:
    # Very High: more than one field is 'high'
    if high_count > 1:
        return "Very High"
    # High: one field is 'high' (and the rest are 'low' or 'moderate')
    elif high_count == 1:
        return "High"
    # Moderate: at least one 'moderate' and no 'high'
    elif moderate_count > 0:
        return "Moderate"
    # Low: all fields are 'low'
    else:
        return "Low"

# Assuming df is your DataFrame, create the new column:
df['RD_DENS_COMB_CLS'] = df.apply(classify_row, axis=1)

In [10]:
df = df[['ASSESSMENT_UNIT_SOURCE_ID','RD_DENS_COMB_CLS']]

In [ ]:
# populate the AGOL layer with new Road Density values
AGO_HOST = 'https://governmentofbc.maps.arcgis.com'
AGO_USERNAME_DSS = 'XXX'
AGO_PASSWORD_DSS = 'XXX'
gis= GIS(AGO_HOST, AGO_USERNAME_DSS, AGO_PASSWORD_DSS, verify_cert=False)

agol_item_id = '7326558fb14c476bbfc8b7b6ddb45ada'

feature_layer = gis.content.get(agol_item_id).layers[0]  # Assuming first layer in the item

# Query all features in the layer
print ('Reading records from AGOL layer')
features = feature_layer.query(where="1=1", out_fields="ASSESSMENT_UNIT_SOURCE_ID, RD_DENS_COMB_CLS", return_geometry=False)
features_dict = {f.attributes['ASSESSMENT_UNIT_SOURCE_ID']: f for f in features}

# Update features with new drought sensitivity values
print ('\nUpdating data in batches')
updated_features = []
for index, row in df.iterrows():
    unit_id = row['ASSESSMENT_UNIT_SOURCE_ID']
    if unit_id in features_dict:
        feature = features_dict[unit_id]
        feature.attributes['RD_DENS_COMB_CLS'] = row['RD_DENS_COMB_CLS']
        updated_features.append(feature)

# Function to batch updates
def batch_update(features, batch_size=100):
    for i in range(0, len(features), batch_size):
        batch = features[i:i + batch_size]
        response = feature_layer.edit_features(updates=batch)
        print(f"..batch {i // batch_size + 1}: Update response:", response['updateResults'][0])

# Batch and update
if updated_features:
    batch_update(updated_features, batch_size=100)  # Adjust batch size as needed
else:
    print("No features updated.")
